<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="45%"><img src="../media/Univ-Utah.jpeg"><br>
</td>
    <td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">University of Utah<br>Population Health Sciences<br>Data Science Workshop</font></h1></td>
<td valign="center" align="center" width="45%"><img
src="../media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>


In [2]:
from helpers import *
import pandas as pd

# Introduction to SQL
Now that we have some background about what databases are and how they're structured, we'll get some hands-on experiencing joining tables and querying data from MIMIC.

## Connecting to MIMIC
Throughout this class, we'll use the function `connect_to_mimic` to connect to the MIMIC database (imported as part of the `helpers` module). This requires the package `pymysql`, so you may have to install that first. 


#### TODO
In the cells below, install `pymysql` and then run the function to connect to the database. Ask your instructor for the password when prompted.

In [2]:
!pip install pymysql

In [3]:
conn = connect_to_mimic("uu-phs")
conn

Enter password for MIMIC2 database········


Great, we've connected to MIMIC! Now we're almost ready to pull some data. But first we need to learn some basic SQL.

## Administrative and demographic data
For next couple of notebooks, we'll focus on tables containing **ddministrative and demographic data**. These tables define general information about the patient or their hospitalizations. This includes data elements such as:
- Name
- Sex
- Date of birth
- Insurance information
- Admit/discharge datetime

We'll focus on three tables for now: `admission`, `d_patients`, and `demographic_detail`. Let's start writing some queries in SQL!

## SQL
**Structured Query Language (SQL)** is a programming language used to interact with many relational databases. There are many different *flavors* of SQL that vary slightly from one another, but the core logic is typically the same.

When we use SQL, we execute a **query** which runs some logic to specifyl, filter, and transform data in the database. It then returns the **result set** to us. In Python, we can use the `pandas` function `read_sql` to connect to a database and execute a query.  `pd.read_sql` takes two required arguments:

- `sql`: A string containing a SQL query
- `con`: The connection object which allows us to access the database 
For the second argument, we'll use the `conn` object returned by `connect_to_mimic`. The first argument should be a string containing SQL code.

Here is an example of executing a query:

In [14]:
query = """
SELECT *
FROM admissions
WHERE admit_dt <= '3033-07-08 00:00:00'
LIMIT 10;
"""

pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,hadm_id,subject_id,admit_dt,disch_dt
0,2,24807,3033-07-08 00:00:00,3033-07-17 00:00:00
1,10,14884,3015-08-28 00:00:00,3015-09-05 00:00:00
2,49,2328,2549-07-12 00:00:00,2549-07-16 00:00:00
3,50,6335,2789-08-21 00:00:00,2789-09-01 00:00:00
4,68,7664,2535-08-01 00:00:00,2535-08-10 00:00:00
5,74,19937,2861-08-04 00:00:00,2861-08-10 00:00:00
6,80,1984,2575-07-06 00:00:00,2575-07-17 00:00:00
7,123,12792,2882-06-26 00:00:00,2882-08-05 00:00:00
8,130,14212,2966-07-14 00:00:00,2966-08-07 00:00:00
9,173,17287,2985-05-07 00:00:00,2985-05-17 00:00:00


## Structure of a SQL query

Let's go back through that SQL query. If we were to translate the query to natural language, we might express it as:

---
**"Give me the top 10 rows of data from the `admissions` table where the admit datetime was before July 8th, 3033."** (Why do you think the dates look so weird?)

---

This returns a pandas DataFrame with four columns and 10 rows.

#### Discussion
What columns are returned by this query? What do the columns/values mean?

</br>
More generally, here is the structure of a SQL query:

--- 
<html>
<font>
    <p style="font-family:courier";>SELECT COLUMN NAMES (or *)</p>
    <p style="font-family:courier";> FROM TABLE NAME</p>
    <p style="font-family:courier";>(optional) JOIN</p>
    <p style="font-family:courier";>WHERE (condition)</p>
    <p style="font-family:courier";>(optional) ORDER BY ...</p>
    <p style="font-family:courier";>(optional) LIMIT N ...</p>
</html>
---

Here is a quick explanation of each clause (we'll go through each in detail later):
- `SELECT`: This tells us which columns we want to pull. If we say `SELECT *`, that means `"SELECT ALL"`
- `FROM`: This specifies which table the data will be in
- `JOIN`: This joins two tables together using a common key. If we only need a single table, we can leave this out.
- `WHERE`: This allows to filter to where rows where a certain condition is matched
- `ORDER BY`: This sorts the rows by a particular column
- `LIMIT`: This means we only want the first `N` rows. In this class, we'll typically use this clause so we don't pull excessively large datasets.

    
#### TODO

In [5]:
# RUN CELL TO SEE QUIZ
quiz_simple_query_parts

Here is a slightly more complicated query:

In [6]:
query = """
SELECT a.hadm_id, a.subject_id, a.admit_dt, a.disch_dt, p.sex, p.dob, p.dod, p.hospital_expire_flg
FROM d_patients  p
    INNER JOIN admissions a
        ON p.subject_id = a.subject_id
WHERE hospital_expire_flg = 'Y'
ORDER BY dod
LIMIT 10;
"""

pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,hadm_id,subject_id,admit_dt,disch_dt,sex,dob,dod,hospital_expire_flg
0,5756,17754,2501-04-07 00:00:00,2501-04-08 00:00:00,F,2433-06-15 00:00:00,2501-04-08 00:00:00,Y
1,6311,17754,2501-03-18 00:00:00,2501-04-01 00:00:00,F,2433-06-15 00:00:00,2501-04-08 00:00:00,Y
2,14290,24082,2501-04-28 00:00:00,2501-05-01 00:00:00,F,2445-02-19 00:00:00,2501-05-01 00:00:00,Y
3,7627,8907,2501-11-22 00:00:00,2501-12-06 00:00:00,M,2429-06-06 00:00:00,2501-12-06 00:00:00,Y
4,30624,27327,2502-01-09 00:00:00,2502-01-12 00:00:00,F,2411-09-29 00:00:00,2502-01-12 00:00:00,Y
5,6687,22338,2502-04-12 00:00:00,2502-04-19 00:00:00,M,2426-05-24 00:00:00,2502-04-19 00:00:00,Y
6,2485,871,2502-04-01 00:00:00,2502-05-05 00:00:00,F,2420-10-30 00:00:00,2502-05-05 00:00:00,Y
7,22765,11724,2503-02-17 00:00:00,2503-02-26 00:00:00,M,2422-05-26 00:00:00,2503-02-26 00:00:00,Y
8,34287,29348,2503-05-03 00:00:00,2503-05-28 00:00:00,F,2444-09-04 00:00:00,2503-05-28 00:00:00,Y
9,5716,1795,2501-11-14 00:00:00,2501-11-30 00:00:00,F,2455-12-23 00:00:00,2503-07-24 00:00:00,Y


#### TODO

In [15]:
# RUN CELL TO SEE QUIZ
quiz_tables_in_query

In [16]:
# RUN CELL TO SEE QUIZ
quiz_order_by_column

## Writing queries
Now let's go through the different parts of a SQL query and get some practice writing our own queries.

### The essentials

There are two components that all of our SQL queries will have:
1. **The `SELECT` statement**: Here, we *select* the columns that we want to retrieve from the database. You can either list the specific columns you want separated or commas or just say **"*"** to pull all of them
2. **The `FROM` statement**: We need to specify what table these columns are coming from. We sometimes give a table name an "alias" (often a single letter) to refer to in the query.

While it's not always essential, in this class we will also often have:
3. **A `LIMIT`** statement**: Limit the number of rows we're pulling so we don't overwhelm the database (or your machine).

So a very basic query could just select all (or some) of the rows and all of the columns from a single table. The following query pulls every column from the first 10 rows of **d_patients** while giving the table an alias of `d`:  

In [9]:
query = """
SELECT *
FROM d_patients d
LIMIT 10;
"""

pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,sex,dob,dod,hospital_expire_flg
0,3,M,2606-02-28 00:00:00,2683-05-02 00:00:00,N
1,12,M,2803-05-14 00:00:00,2875-10-09 00:00:00,Y
2,21,M,3051-05-22 00:00:00,3139-03-28 00:00:00,Y
3,26,M,3007-03-02 00:00:00,3080-12-22 00:00:00,N
4,31,M,2606-05-17 00:00:00,2678-08-29 00:00:00,Y
5,37,M,3195-09-11 00:00:00,3265-12-31 00:00:00,N
6,56,F,2553-05-26 00:00:00,2644-01-23 00:00:00,Y
7,61,M,3297-10-27 00:00:00,3353-02-09 00:00:00,Y
8,67,M,2903-06-04 00:00:00,2976-11-29 00:00:00,Y
9,78,M,2729-08-08 00:00:00,2781-03-11 00:00:00,N


#### TODO
Write and execute a query to select just the `hadm_id`, `subject_id`, `admit_dt`, and `disch_dt` from the `admissions` table. Give the table an alias of `a`. Limit to the first 25 rows. Save the result as `df`.

In [10]:
query = """
SELECT hadm_id, subject_id, admit_dt, disch_dt
FROM admissions
LIMIT 25;
"""

df = pd.read_sql(query, conn)
df

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,hadm_id,subject_id,admit_dt,disch_dt
0,2,24807,3033-07-08 00:00:00,3033-07-17 00:00:00
1,3,7675,3388-05-16 00:00:00,3388-05-21 00:00:00
2,6,23547,3381-04-03 00:00:00,3381-04-22 00:00:00
3,10,14884,3015-08-28 00:00:00,3015-09-05 00:00:00
4,12,8652,3125-09-11 00:00:00,3125-09-22 00:00:00
5,15,7728,3491-04-08 00:00:00,3491-04-16 00:00:00
6,34,17025,3118-10-14 00:00:00,3118-10-25 00:00:00
7,35,15784,3036-10-02 00:00:00,3036-10-07 00:00:00
8,45,23014,3108-05-01 00:00:00,3108-06-05 00:00:00
9,47,9129,3129-06-19 00:00:00,3129-06-25 00:00:00


In [11]:
set(df.columns) == set(['hadm_id', 'subject_id', 'admit_dt', 'disch_dt'] )

True

In [12]:
# query = """
# SELECT hadm_id, ____, admit_dt, disch_dt
# ____ admissions
# LIMIT ____;
# """

# pd.read_sql(query, conn)

In [13]:
test_query_result1.test(df)

That is correct!


## Joining tables
In a relational database like MIMIC, different attributes for entities are stored in different tables. These disparate tables can then be joined together in a query using a `join` statement. The column `subject_id`, which is the identifier for a patient, is consistent between these two columns and can be used to join them together:

```SQL
FROM table1
    INNER JOIN table2
        ON table1.column = table2.column
```

*Note*: We'll talk more about the phrase `INNER JOIN` later, as well as other types of joins.

#### TODO
Join the `demographic_detail` and `d_patients` tables using the `subject_id` column in both as the joining keys. Select all columns and the **top 10** rows.

In [14]:
query = """
SELECT *    -- Select statement
FROM demographic_detail d
    INNER JOIN d_patients p
        ON d.subject_id = p.subject_id
"""

In [15]:
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,hadm_id,marital_status_itemid,marital_status_descr,ethnicity_itemid,ethnicity_descr,overall_payor_group_itemid,overall_payor_group_descr,religion_itemid,religion_descr,admission_type_itemid,admission_type_descr,admission_source_itemid,admission_source_descr,subject_id,sex,dob,dod,hospital_expire_flg
0,56,28766,NaN,None,200083,WHITE,200053,MEDICARE-PRIVATE,200060.0,NOT SPECIFIED,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT,56,F,2553-05-26 00:00:00,2644-01-23 00:00:00,Y
1,37,18052,200050.0,MARRIED,200083,WHITE,200053,MEDICARE-PRIVATE,200047.0,JEWISH,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT,37,M,3195-09-11 00:00:00,3265-12-31 00:00:00,N
2,78,15161,200071.0,SEPARATED,200018,BLACK/AFRICAN AMERICAN,200052,MEDICARE,200060.0,NOT SPECIFIED,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT,78,M,2729-08-08 00:00:00,2781-03-11 00:00:00,N
3,67,35878,200072.0,SINGLE,200083,WHITE,200052,MEDICARE,200047.0,JEWISH,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT,67,M,2903-06-04 00:00:00,2976-11-29 00:00:00,Y
4,3,2075,200050.0,MARRIED,200083,WHITE,200052,MEDICARE,200023.0,CATHOLIC,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT,3,M,2606-02-28 00:00:00,2683-05-02 00:00:00,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5069,32667,36071,NaN,None,200083,WHITE,200052,MEDICARE,200062.0,OTHER,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT,32667,M,2778-01-19 00:00:00,2866-02-27 00:00:00,Y
5070,32627,33957,200050.0,MARRIED,200083,WHITE,200052,MEDICARE,200047.0,JEWISH,200028,EMERGENCY,200074,TRANSFER FROM HOSP/EXTRAM,32627,M,2699-08-04 00:00:00,2788-11-14 00:00:00,Y
5071,32675,35514,200071.0,SEPARATED,200018,BLACK/AFRICAN AMERICAN,200051,MEDICAID,200060.0,NOT SPECIFIED,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT,32675,M,2758-07-07 00:00:00,2816-04-29 00:00:00,Y
5072,32686,30102,200050.0,MARRIED,200083,WHITE,200067,PRIVATE,200081.0,UNOBTAINABLE,200028,EMERGENCY,200074,TRANSFER FROM HOSP/EXTRAM,32686,M,2543-12-06 00:00:00,2612-06-27 00:00:00,Y


## Filtering results
Typically we don't want to return *all* rows from a table. We instead usually filter based on conditions related to the columns of the table. This is where the `WHERE` clause comes in.

For example, to get the demographic details for a single patient, we can filter based on the `subject_id` column:

In [16]:
query = """
SELECT *
FROM demographic_detail d
WHERE subject_id = 78
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,hadm_id,marital_status_itemid,marital_status_descr,ethnicity_itemid,ethnicity_descr,overall_payor_group_itemid,overall_payor_group_descr,religion_itemid,religion_descr,admission_type_itemid,admission_type_descr,admission_source_itemid,admission_source_descr
0,78,15161,200071,SEPARATED,200018,BLACK/AFRICAN AMERICAN,200052,MEDICARE,200060,NOT SPECIFIED,200028,EMERGENCY,200029,EMERGENCY ROOM ADMIT


You can also use standard comparators like `!=`, `>`, `>=`, `<`, and `<=`. 

Run the query below - it returns an error. Scroll to the bottom of the error traceback and read the error description:

`"Column 'subject_id' in where clause is ambiguous"`


In [17]:
query = """
SELECT p.subject_id, p.dob, p.dod, d.admission_type_descr
FROM demographic_detail d INNER JOIN
    d_patients p
        ON d.subject_id = p.subject_id
WHERE subject_id = 78
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql '
SELECT p.subject_id, p.dob, p.dod, d.admission_type_descr
FROM demographic_detail d INNER JOIN
    d_patients p
        ON d.subject_id = p.subject_id
WHERE subject_id = 78
': (1052, "Column 'subject_id' in where clause is ambiguous")

#### TODO

In [18]:
# RUN CELL TO SEE QUIZ
MultipleChoiceQuiz("What do you think the error above means?",
                  answer="More than one table has a column subject_id",
                  options=["There is no column named subject_id", "There's no row where subject_id = 78"])

When more than one table in our query has a column of the same name, we need to specify which table we're referring to. We can specify a column using the notation:

`table_name.column_name`

or, if we're using aliases::

`alias.column_name`

`WHERE d.subject_id = 78`

or:

`WHERE p.subject_id = 78`

#### TODO
Which of the following changes to the `WHERE` clause would cause our query to run correctly?
- **a)** `WHERE d.subject_id = 78`
- **b)** `WHERE p.subject_id = 78`
- **c)** `WHERE ANY(subject_id) = 78`

In [19]:
# RUN CELL TO SEE QUIZ
quiz_fix_where_ambiguity

#### TODO
Based on your answer to the previous quiz, fix the query and rerun it, saving the result as `df`.

In [20]:
query = """
SELECT p.subject_id, p.dob, p.dod, d.admission_type_descr
FROM demographic_detail d INNER JOIN
    d_patients p
        ON d.subject_id = p.subject_id
WHERE p.subject_id = 78
"""
df = pd.read_sql(query, conn)
df

,subject_id,dob,dod,admission_type_descr
0,78,2729-08-08 00:00:00,2781-03-11 00:00:00,EMERGENCY


In [22]:
# query = """

# """
# df = pd.read_sql(query, conn)
# df

In [23]:
# RUN CELL TO TEST VALUE
test_fixed_where_ambiguity.test(df)

That is correct!


## Ordering results
Finally, we can order the queried data by using the `ORDER BY` clause:

In [24]:
query = """
SELECT *
FROM d_patients
WHERE subject_id IN (56, 78, 37)
ORDER BY subject_id;
"""
pd.read_sql(query, conn)

,subject_id,sex,dob,dod,hospital_expire_flg
0,37,M,3195-09-11 00:00:00,3265-12-31 00:00:00,N
1,56,F,2553-05-26 00:00:00,2644-01-23 00:00:00,Y
2,78,M,2729-08-08 00:00:00,2781-03-11 00:00:00,N


By default, `ORDER BY` sorts values in **ascending** order. But we can switch to **descending** order using the `DESC` keyword:

```sql
ORDER BY column DESC
```

#### TODO
Change the query above to sort the data by `dob` in *descending* order. Rerun and save as `df`.

In [25]:
query = """
SELECT *
FROM d_patients
WHERE subject_id IN (56, 78, 37)
ORDER BY dob DESC;
"""
df = pd.read_sql(query, conn)
df

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,sex,dob,dod,hospital_expire_flg
0,37,M,3195-09-11 00:00:00,3265-12-31 00:00:00,N
1,78,M,2729-08-08 00:00:00,2781-03-11 00:00:00,N
2,56,F,2553-05-26 00:00:00,2644-01-23 00:00:00,Y


In [26]:
# query = """

# """
# df = pd.read_sql(query, conn)
# df

In [27]:
def test_query_dob_descending(actual):
    import pandas as pd
    if not isinstance(actual, pd.DataFrame):
        print(f"Incorrect. df should be a pandas DataFrame, not {type(actual)}")
        return
    if len(actual) != 3:
        print(f"Incorrect. df should have 3 rows. Your dataframe had {len(actual)}.")
        return
    if set(actual.columns) != {'subject_id', 'sex', 'dob', 'dod', 'hospital_expire_flg'}:
        print(
            f"Incorrect. Your dataframe should have columns ['subject_id', 'sex', 'dob', 'dod', 'hospital_expire_flg'], not {list(actual.columns)}")
        return
    if set(actual["subject_id"]) != {37, 78, 56}:
        print(f"Incorrect. Your dataframe should have subject_id's (37, 78, 56), not {tuple(actual['subject_id'])}")
        return
    if tuple(actual["subject_id"]) != (37, 78, 56):
        print(f"Incorrect. Your dataframe should have subject ids in the order (37, 78, 56), not {tuple(actual['subject_id'])}")
    
    print("That is correct!")
test_query_dob_descending = ValueTest(validation_func=test_query_dob_descending)

In [28]:
# RUN CELL TO TEST VALUE
test_query_dob_descending.test(df)

That is correct!


## Renaming columns
Sometimes we might want to rename our columns, maybe to make it a name that's easier to understand or that is less ambiguous. We do this the same way we assigned *aliases* to tables:

```sql
SELECT column1 AS new_name
    ,column2 new_name2 -- 'AS' is optional
```

#### TODO
Select the first 10 rows of `d_patients` and rename `dob` to `date_of_birth` and `dod` to `date_of_death`. Save the assignment as `df_patients_renamed`

In [4]:
query = """
SELECT
    subject_id
    ,sex
    ,dob AS date_of_birth
    ,dod AS date_of_death
    ,hospital_expire_flg
FROM d_patients
LIMIT 10
"""

df_patients_renamed = pd.read_sql(query, conn)
df_patients_renamed

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,sex,date_of_birth,date_of_death,hospital_expire_flg
0,3,M,2606-02-28 00:00:00,2683-05-02 00:00:00,N
1,12,M,2803-05-14 00:00:00,2875-10-09 00:00:00,Y
2,21,M,3051-05-22 00:00:00,3139-03-28 00:00:00,Y
3,26,M,3007-03-02 00:00:00,3080-12-22 00:00:00,N
4,31,M,2606-05-17 00:00:00,2678-08-29 00:00:00,Y
5,37,M,3195-09-11 00:00:00,3265-12-31 00:00:00,N
6,56,F,2553-05-26 00:00:00,2644-01-23 00:00:00,Y
7,61,M,3297-10-27 00:00:00,3353-02-09 00:00:00,Y
8,67,M,2903-06-04 00:00:00,2976-11-29 00:00:00,Y
9,78,M,2729-08-08 00:00:00,2781-03-11 00:00:00,N


In [5]:
# query = """
# SELECT
#     subject_id
#     ,sex
#     ,__ AS ____
#     ,____
#     ,hospital_expire_flg
# FROM d_patients
# LIMIT 10
# """

# df_patients_renamed = pd.____(____, conn)
# df_patients_renamed

In [6]:
# RUN CELL TO TEST VALUE
validate_df_patients_renamed.test(df_patients_renamed)

That is correct!


## Creating new columns with new values
Often, tables don't have the exist data element we want. For example, let's say that we want to study patient age at death. There is no exact column for this in `d_patients`, but we can use `dod` and `dob` to calculate a new column.

Just like Python, SQL has certain **functions** that you can add to your queries. One such function is `DATEDIFF` which calculates the number of days between two dates:

```sql
SELECT DATEDIFF(date1, date2)
```

We can use that to calculate the number of days between patients' death and birth dates. When we calculate a new column we need to give it a name, so we'll call this one `age_at_death_days`.

In [7]:
query = """
SELECT 
    subject_id,
    dod,
    DATEDIFF(dod, dob) age_at_death_days
FROM d_patients p
LIMIT 10
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,dod,age_at_death_days
0,3,2683-05-02 00:00:00,28187
1,12,2875-10-09 00:00:00,26446
2,21,3139-03-28 00:00:00,32086
3,26,3080-12-22 00:00:00,26959
4,31,2678-08-29 00:00:00,26402
5,37,3265-12-31 00:00:00,25679
6,56,2644-01-23 00:00:00,33113
7,61,3353-02-09 00:00:00,20193
8,67,2976-11-29 00:00:00,26842
9,78,2781-03-11 00:00:00,18843


We can also do basic arithmetic like addition, subtraction, multiplication, and division using operators similar to Python: `+`, `-`, `*`, `/`.

#### TODO
Make a new version of `df_patients` with all 4,000 rows and a new column called `age_at_death` which is the patient's age when they died *in years*.

In [8]:
# RUN CELL TO SEE HINT
hint_age_in_years

In [11]:
query = """
SELECT *
    ,DATEDIFF(dod, dob) / 365 age_at_death
FROM d_patients;
"""

df_patients = pd.read_sql(query, conn)
df_patients.head()

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,sex,dob,dod,hospital_expire_flg,age_at_death
0,3,M,2606-02-28 00:00:00,2683-05-02 00:00:00,N,77.2247
1,12,M,2803-05-14 00:00:00,2875-10-09 00:00:00,Y,72.4548
2,21,M,3051-05-22 00:00:00,3139-03-28 00:00:00,Y,87.9068
3,26,M,3007-03-02 00:00:00,3080-12-22 00:00:00,N,73.8603
4,31,M,2606-05-17 00:00:00,2678-08-29 00:00:00,Y,72.3342


In [12]:
# query = """
# SELECT *
#     ,____
# """

# df_patients = pd.read_sql(query, conn)
# df_patients.head()

In [13]:
# RUN CELL TO TEST VALUE
test_age_at_death.test(df_patients)

That is correct!


## Aggregating data
So far, everything we've done with SQL has been at the **row-level**. That is, we've written queries that have returned results with a single entity (patient, hospitalization, etc.) per row. Next we'll start looking at how to **aggregate** data in SQL.

Some examples of aggregate data we coud compute in MIMIC include:
- The number of admissions
- The count of patients by sex
- The min and max admission date for patients
- The mean/min/max/standard deviation of length of stay

In `module_2`, we learned how to do many of these calculatins in Python using `pandas`. Aggregating data in SQL is very similar. Each of the calculations described above can be computed using a **SQL function** like `COUNT()`, `MIN()`, or `MAX()`.

### Counts
One of the most basic aggegations is simply counting the number of rows in a table. We can get this by selecting `COUNT(*)`:

In [18]:
query = """
SELECT COUNT(1) AS n
FROM admissions
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,n
0,5074


#### TODO
How would you represent the result above?

In [24]:
# RUN CELL TO SEE QUIZ
quiz_count_n

Aggregate queries can have other clauses like `WHERE`, `JOIN`, etc., so you can filter and join the data you're counting.

#### TODO
How many rows in the table `demographic_detail` represent an admission from the emergency room?

In [29]:
# RUN CELL TO SEE QUIZ
quiz_count_ed_admit

In [30]:
query = """
SELECT COUNT(1)
FROM demographic_detail 
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,COUNT(1)
0,5074


### Mins, Maxes, and Means
SQL has functions to calculate extreme values, means, and standard deviations. For example, the query below calculates the earliest/latest dates of birth and death:

In [31]:
query = """
SELECT 
    MIN(dob) earliest_birth, MAX(dob) latest_birth, MIN(dod) earliest_death, MAX(dod) latest_death
FROM d_patients p
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,earliest_birth,latest_birth,earliest_death,latest_death
0,2405-06-25 00:00:00,3498-02-04 00:00:00,2501-04-08 00:00:00,3586-01-08 00:00:00


We can also pass in transformed values to functions. Earlier we'd seen how to calculate how old a patient was in days or years. Now we can calculate summary statistics like the mean and standard deviation using `AVG()` and `STD()`:

In [32]:
query = """
SELECT 
    AVG(DATEDIFF(dod, dob)) avg_age_at_death_days,
    STD(DATEDIFF(dod, dob)) std_age_at_death_days
FROM d_patients p
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg_age_at_death_days,std_age_at_death_days
0,26353.911,5940.857974


#### TODO
Calculate the min, max, mean, and standard deviation age at death *in years*.

In [39]:
quiz_summary_stats_age_years = FreeTextTest("Enter the min, max, mean, and standard deviations of age at death in years. </br>Round to the nearest whole number and enter each separated by a comma (e.g., '10,45,22,3')",
                                           answer="0,105,72,16")
quiz_summary_stats_age_years

In [36]:
query = """
SELECT 
    MIN(DATEDIFF(dod, dob)/365) min_age_at_death,
    MAX(DATEDIFF(dod, dob)/365) max_age_at_death,
    AVG(DATEDIFF(dod, dob)/365) mean_age_at_death,
    STD(DATEDIFF(dod, dob)/365) sd_age_at_death
FROM d_patients p
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,min_age_at_death,max_age_at_death,mean_age_at_death,sd_age_at_death
0,0.0,105.4219,72.202496,16.276323


### GROUP BY
The queries above gave us single aggregate stats over an entire set of patients. But we might want to break our statistics up into groups. We'll use the `GROUP BY` clause for that. 

The `GROUP BY` clause tells us which column to use for breaking our patients up into groups. This works just like `df.groupby` in pandas. 

The query below counts the number of patients by `sex`:

In [40]:
query = """
SELECT sex, COUNT(1) n
FROM d_patients 
GROUP BY sex
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sex,n
0,None,13
1,F,1856
2,M,2131


#### TODO
Count the number of hospital admissions grouped by `admission_source_descr`.

In [44]:
# RUN CELL TO SEE QUIZ
quiz_count_admission_source

In [43]:
query = """
SELECT admission_source_descr, COUNT(1)
FROM demographic_detail 
GROUP BY admission_source_descr
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,admission_source_descr,COUNT(1)
0,** INFO NOT AVAILABLE **,4
1,CLINIC REFERRAL/PREMATURE,211
2,EMERGENCY ROOM ADMIT,3193
3,HMO REFERRAL/SICK,1
4,PHYS REFERRAL/NORMAL DELI,616
5,TRANSFER FROM HOSP/EXTRAM,957
6,TRANSFER FROM OTHER HEALT,13
7,TRANSFER FROM SKILLED NUR,79


We can also group by multiple columns at once. This query calculates the number of patients in `demographic_detail` grouped by both sex and race and sorts in descending order of count.

In [51]:
pd.read_sql("select * from d_patients limit 10", conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_id,sex,dob,dod,hospital_expire_flg
0,3,M,2606-02-28 00:00:00,2683-05-02 00:00:00,N
1,12,M,2803-05-14 00:00:00,2875-10-09 00:00:00,Y
2,21,M,3051-05-22 00:00:00,3139-03-28 00:00:00,Y
3,26,M,3007-03-02 00:00:00,3080-12-22 00:00:00,N
4,31,M,2606-05-17 00:00:00,2678-08-29 00:00:00,Y
5,37,M,3195-09-11 00:00:00,3265-12-31 00:00:00,N
6,56,F,2553-05-26 00:00:00,2644-01-23 00:00:00,Y
7,61,M,3297-10-27 00:00:00,3353-02-09 00:00:00,Y
8,67,M,2903-06-04 00:00:00,2976-11-29 00:00:00,Y
9,78,M,2729-08-08 00:00:00,2781-03-11 00:00:00,N


In [57]:
query = """
SELECT 
    d.ethnicity_descr, 
    p.sex,
    COUNT(*) n
FROM d_patients p
    INNER JOIN demographic_detail d
        ON p.subject_id = d.subject_id
GROUP BY d.ethnicity_descr, p.sex
ORDER BY COUNT(*) desc
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ethnicity_descr,sex,n
0,WHITE,M,1898
1,WHITE,F,1634
2,UNKNOWN/NOT SPECIFIED,M,399
3,UNKNOWN/NOT SPECIFIED,F,330
4,BLACK/AFRICAN AMERICAN,F,268
5,BLACK/AFRICAN AMERICAN,M,206
6,ASIAN,M,54
7,OTHER,M,49
8,HISPANIC OR LATINO,M,41
9,PATIENT DECLINED TO ANSWER,M,38


#### TODO
Write a query to count how many patients died in the hospital grouped by `sex`. Then answer the quiz below.

In [63]:
# RUN CELL TO SEE HINT
hint_count_hospital_expire_by_sex

In [73]:
# RUN CELL TO SEE QUIZ
quiz_count_hospital_expire_by_sex

In [71]:
query = """
SELECT sex, hospital_expire_flg, COUNT(1) n
FROM d_patients
WHERE hospital_expire_flg = 'Y'
    and SEX != 'None'
GROUP BY sex, hospital_expire_flg
"""
pd.read_sql(query, conn)

/Users/alecchapman/opt/miniconda3/envs/phs/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sex,hospital_expire_flg,n
0,F,Y,809
1,M,Y,841
